In [2]:
import os
import pandas as pd
import gensim
import nltk
import pickle

from gensim.utils import simple_preprocess 
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
stemmer = PorterStemmer()

In [12]:
desktop = os.path.join(os.path.expanduser('~'), 'Desktop')
large_file = os.path.join(desktop, 'large_file')
processed_wiki_sub = os.path.join(large_file, 'processed_wiki_sub')
postfix = 'abcdefghijklmno'
text_path_prefix = os.path.join(processed_wiki_sub, 'processed_wiki.texta')

In [13]:
def preprocess(text):
    result = []
    # convert document into list of lowercase tokens, filter based on token length
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(token)
    return result


In [14]:
dictionary = gensim.corpora.Dictionary()
bow_corpus = []
for letter in postfix:
    text_path = text_path_prefix+letter
    with open(text_path, 'r') as file:
        text = file.read()
    line = text.split('\n')
    df = pd.DataFrame(line, columns = ['article'])
    print("adding articles from {}".format(text_path))
    processed_docs = df['article'].map(preprocess)
    print("wiki subpart added~")
    #print("sample article: {}, added!".format(processed_docs[0]))
    dictionary.add_documents(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus += [dictionary.doc2bow(doc) for doc in processed_docs] # bow rep for each article

adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textaa
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textab
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textac
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textad
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textae
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textaf
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textag
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_sub/processed_wiki.textah
wiki subpart added~
adding articles from /Users/i355904/Desktop/large_file/processed_wiki_su

In [15]:
dictionary.save('dictionary.complete')

In [3]:
with open('bow_corpus_complete.pkl','wb') as file:
    pickle.dump(bow_corpus,file)

NameError: name 'bow_corpus' is not defined

In [4]:
from gensim.models import LdaMulticore
lda_model = LdaMulticore(bow_corpus, num_topics=300, id2word=dictionary, passes=2, workers=8)

NameError: name 'bow_corpus' is not defined

In [ ]:
lda_model.save('model_complete/lda_full_corpus.model')

In [184]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.010*"puerto" + 0.007*"rico" + 0.005*"univers" + 0.004*"alabama" + 0.004*"citi" + 0.003*"counti" + 0.003*"rican" + 0.003*"world" + 0.003*"star" + 0.003*"nation"
Topic: 1 
Words: 0.016*"bridg" + 0.011*"citi" + 0.008*"canal" + 0.005*"build" + 0.004*"river" + 0.002*"road" + 0.002*"nation" + 0.002*"construct" + 0.002*"develop" + 0.002*"south"
Topic: 2 
Words: 0.005*"test" + 0.004*"club" + 0.003*"cricket" + 0.003*"play" + 0.003*"australia" + 0.003*"centuri" + 0.003*"match" + 0.003*"nation" + 0.002*"record" + 0.002*"world"
Topic: 3 
Words: 0.012*"citi" + 0.009*"church" + 0.006*"centuri" + 0.006*"town" + 0.006*"counti" + 0.005*"popul" + 0.005*"french" + 0.004*"german" + 0.004*"municip" + 0.004*"build"
Topic: 4 
Words: 0.006*"john" + 0.005*"william" + 0.005*"bear" + 0.005*"king" + 0.005*"english" + 0.005*"henri" + 0.004*"georg" + 0.004*"compos" + 0.003*"princ" + 0.003*"charl"
Topic: 5 
Words: 0.004*"wear" + 0.003*"design" + 0.003*"form" + 0.003*"power" + 0.003*"coin" + 0.003*

In [185]:
lda_model.show_topic(0)

[('puerto', 0.009830458),
 ('rico', 0.0070977216),
 ('univers', 0.00476907),
 ('alabama', 0.0039931443),
 ('citi', 0.0038465243),
 ('counti', 0.0032254884),
 ('rican', 0.0029379986),
 ('world', 0.002667393),
 ('star', 0.002613539),
 ('nation', 0.002608398)]

In [186]:
test_set = os.path.join(processed_wiki_sub,'processed_wiki.textab')
with open(test_set, 'r') as file2:
    text = file2.read()
line = text.split('\n')    
df2 = pd.DataFrame(line, columns=['article'])
df2.head()

,article
0,dentary agriculture instead a part of delibera...
1,yuyao is county level city in the northeast of...
2,liu huaqing october january wa general of the ...
3,hui pan nationalism refers to the common ident...
4,man carrying shovel shovel is tool for digging...


In [198]:
test_doc = df2.iloc[0]
test_doc = test_doc['article']
test_doc

'dentary agriculture instead a part of deliberate policy by the qing to enfeeble them life in the russian empire after the exodus the kalmyks that remained part of the russian empire became under the control of the russian empire they however continued their nomadic pastoral lifestyle ranging the pasture between the don and the volga river wintering in the lowland along the shore of the caspian sea a far a sarpa lake to the northwest and lake manych gudilo to the west in the spring they moved along the don river and the sarpa lake system attaining the higher ground along the don in the summer passing the autumn in the sarpa and volga lowland in october and november they returned to their winter camp and pasture krader citing pallas vol despite their great loss in population the torghut still remained the numerically superior and dominating the kalmyks the other kalmyks in russia included oirats and khoshut element of the choros and khoit also were present but were too few in number to 

In [199]:
bow_vector = dictionary.doc2bow(preprocess(test_doc))
lda_model[bow_vector]

[(21, 0.03768808),
 (24, 0.016039494),
 (33, 0.08597957),
 (44, 0.25151387),
 (51, 0.26032808),
 (62, 0.015545914),
 (91, 0.32431272)]

In [200]:
'''
Pickle large files so they can be reload in future
'''

# import pickle

# with open('processed_docs.pkl', 'wb') as file:
#     pickle.dump(processed_docs, file)
# lda_model.save('lda.model')

# with open('dictionary.pkl', 'wb') as file:
#     pickle.dump(dictionary, file)
# with open('bow_corpus.pkl', 'wb') as file:
#     pickle.dump(bow_corpus, file)

In [202]:
len(bow_corpus)

102100